# 필요한 라이브러리 불러오기

In [ ]:
import sys, os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

# 데이터 정의

In [ ]:
csv_list = 'helios1_1980.csv'
parameter = ['r_sun','n_p','Bx','Tp_par','Tp_perp','T','A','v','new_alf']

os.chdir('C:/Users/hjm01/Desktop/겨울방학/alfven')
Data = pd.read_csv(csv_list,index_col='Time',parse_dates=True)[parameter]

## 데이터 샘플링: 중복되는 포인트 제거

In [ ]:
period = '10T'

Data = Data.resample(period).mean()

# 변수 정의

In [ ]:
e = 4.8*10**(-10) ; m = 1.672*10**(-24) ; k = 1.381*10**(-16)

start_time = datetime(1980,5,14,0,0,0)

Data['ex_t'] = (1.5*10**8)*Data['r_sun']/Data['v']

Data['logarithm'] = np.log((3*np.sqrt(k*Data['T'])**3)/(2*e**3*np.sqrt(Data['n_p']*np.pi)))

up = 4*np.sqrt(np.pi)*e**4*Data['n_p']*Data['logarithm']
down = 3*np.sqrt(m)*np.sqrt(k*Data['T'])**3

Data['chandran_fre'] = up/down

Data['c_n'] = Data['ex_t']*Data['chandran_fre']

# Type별 Stream List Up

In [ ]:
os.chdir('C:/Users/hjm01/Desktop/겨울방학/alfven/type')
B = pd.read_csv('type_'+str(csv_list),index_col=[0],parse_dates=True)

zip_list = list(zip(B.index,B['type']))

fig, ax = plt.subplots(6,1,sharex=True,figsize=(12,16),constrained_layout=False)
plt.subplots_adjust(hspace=0)

# Alfvenicity 음영 효과

In [ ]:
hour = 6
for i,j in zip_list:
  if j == 0:
    ax[5].axvspan(i,i+timedelta(hours=hour),alpha=0.2,facecolor='red')
  elif j == 1:
    ax[5].axvspan(i,i+timedelta(hours=hour),alpha=0.2,facecolor='blue')
  else:
    pass

# 그래프 Plot

In [ ]:
line1 = ax[0].plot(Data['v'],color='black',linewidth=1,label=r'$V_{sw}$')

ax0 = ax[0].twinx()
line2 = ax0.plot(Data['Bx'],color='blue',linewidth=1,label=r'$B_{r}$')
ax0.set_yticks([-50,0,50,100])
ax0.set_yticklabels(["-50","0","50","100"],fontsize=15)

ax[0].set_zorder(ax0.get_zorder() + 10)
ax[0].patch.set_visible(False)

ax[0].set_ylim(200,900)
ax[0].set_yticks([400,600,800])
ax[0].set_yticklabels(['400','600','800'],fontsize=15)
ax[0].set_ylabel(r'$km/s,$'+'\n'+'nT',fontsize=20,rotation=0,labelpad=40,y=0.3)
ax[0].grid(axis='both',linestyle=':')
leg = ax[0].legend(line1+line2,[l.get_label() for l in line1+line2],framealpha=0.3,handlelength=1,fontsize=20,edgecolor='white')
leg_lines = leg.get_lines() ; plt.setp(leg_lines, linewidth=3)

ax[1].plot(Data['n_p'],color='black',linewidth=1,label=r'$n_p$')
ax[1].set_yscale('log') ; ax[1].set_ylim(5,200)
ax[1].grid(linestyle=":")
ax[1].set_yticks([10,100])
ax[1].set_yticklabels(["10","100"],fontsize=15)
ax[1].set_ylabel(r'$cm^{-3}$',fontsize=20,rotation=0,labelpad=25)
ax[1].legend(framealpha=0.3,handlelength=1,fontsize=20,edgecolor='white',loc='lower left')
ax[1].minorticks_off()

ax[2].plot(Data['Tp_perp']/10**6,color='red',linewidth=1,label=r'$T_{\perp}$',zorder=1)
ax[2].plot(Data['Tp_par']/10**6,color='blue',linewidth=1,label=r'$T_{\parallel}$',zorder=0)
ax[2].set_ylim(0.01,5)
ax[2].set_yscale('log')
ax[2].set_yticks([0.01,0.1,1])
ax[2].set_yticklabels(["0.01","0.1","1"],fontsize=15)
ax[2].grid(axis='both',linestyle=":")
ax[2].minorticks_off()
leg4 = ax[2].legend(framealpha=0.3,handlelength=1,fontsize=20,edgecolor='white',loc='upper left')
plt.setp(leg4.get_lines(), linewidth=3)
ax[2].set_ylabel('MK',fontsize=20,rotation=0,labelpad=70)
ax[2].yaxis.set_label_position("left") ; ax[2].yaxis.tick_right()

ax[3].plot(Data['A'],color='black',linewidth=1)
ax[3].set_ylim(0.5,5)
ax[3].set_yscale('log')
ax[3].set_yticks([0.5,1,5])
ax[3].set_yticklabels(["0.5","1","5"],fontsize=15)
ax[3].grid(axis='x',linestyle=":")
ax[3].set_ylabel(r'$\frac{T_{\perp}}{T_{\parallel}}$',fontsize=30,rotation=0,labelpad=35,y=0.3)
ax[3].axhline(1,color='red',linestyle='--',linewidth=1,zorder=2)
ax[3].minorticks_off()

ax[4].plot(Data['c_n'],color='black',linewidth=1,label=r'$A_{c}$')
ax[4].set_ylim(0.005,10)
ax[4].set_yscale('log')
ax[4].set_ylabel(r'$A_{c}$',fontsize=25,rotation=0,labelpad=60)
ax[4].set_yticks([0.01,0.1,1])
ax[4].set_yticklabels(["0.01","0.1","1"],fontsize=15)
ax[4].grid(axis='x',linestyle=':')
ax[4].axhline(1,color='red',linestyle='--',linewidth=1,zorder=2)
ax[4].minorticks_off()
ax[4].yaxis.set_label_position("left") ; ax[4].yaxis.tick_right()

ax[5].scatter(Data.resample('30T').mean().index,abs(Data['new_alf'].resample('30T').mean()),color='black',s=2,zorder=2)
ax[5].set_ylim(0,1)
ax[5].set_yticks([0.2,0.4,0.6,0.8])
ax[5].set_yticklabels(['0.2','0.4','0.6','0.8'],fontsize=15)
ax[5].set_ylabel(r'$|\sigma_{c}|$',fontsize=25,rotation=0,labelpad=35)
ax[5].grid(axis='x',linestyle=':')

ax[5].axhline(0.7,color='red',linestyle='--',linewidth=1,zorder=2)

line3 = ax[5].plot(Data['r_sun'],color='grey',label=r"$R[AU]$",zorder=10,linewidth=2)
leg3 = ax[5].legend(fontsize=20,handlelength=1,loc="lower right",edgecolor='white')
leg_lines3 = leg3.get_lines() ; plt.setp(leg_lines3, linewidth=3)


ax[5].set_xlim(start_time,start_time+timedelta(days=35))
ax[0].set_title('Helios 1(1980)',fontsize=30)
plt.setp(ax[5].get_xticklabels(), fontsize=20,rotation=-25)

plt.show()